In [207]:

import json
dictionary={}
hateful_or_not={}
count=0
labels=[]
with open ("../data/train.jsonl",'r') as f:
    for line in f:
        elem=json.loads(line)
        labels.append(int(elem['label']))
        if (int(elem['label'])==0):
            hateful_or_not[int(elem['id'])]=0
        else:
            hateful_or_not[int(elem['id'])]=1
            count+=1
# print(count)
probab_hateful=count/len(hateful_or_not)
probab_hateful_final=probab_hateful
words_captions_hateful={}
words_captions_normal={}
with open("caption.json", 'r') as f:
    dictionary=json.load(f)
    for image_id,elem in dictionary.items():
        # print(image_id)
        for word in elem['sentence'].split(" "):
            # print(hateful_or_not[int(image_id)])
            if (hateful_or_not[int(image_id)]==0):
                if word in words_captions_normal:
                    words_captions_normal[word]+=1
                else:
                    words_captions_normal[word]=1
            else:
                # print(image_id)
                if word in words_captions_hateful:
                    words_captions_hateful[word]+=1
                else:
                    words_captions_hateful[word]=1


# print(words_captions_hateful)
# print(words_captions_normal)
with open("words_captions_hateful.json", 'w') as f:
    json.dump(words_captions_hateful,f)
with open("words_captions_normal.json", 'w') as f:
    json.dump(words_captions_normal,f)
# print(words_captions_hateful)

In [211]:
probab_hateful=1/2
import numpy as np

class NaiveBayesClassifier:
    def __init__(self):
        self.class_prior = {}
        self.word_likelihoods = {}

    def train(self, word_counts, hateful_dict, normal_dict):
        # for val in hatef
        total_hateful=0
        total_normal=0
        for val in hateful_dict.values():
            total_hateful+=val
        for val in normal_dict.values():
            total_normal+=val
        # total_hateful = sum(hateful_counts)
        # total_normal = sum(normal_counts)
        total_examples = total_hateful + total_normal
        
        # self.class_prior[1] = total_hateful / total_examples
        # self.class_prior[0] = total_normal / total_examples
        self.class_prior[1] = probab_hateful
        self.class_prior[0] = 1 - probab_hateful
        
        for word in word_counts:
            hateful_count=0
            normal_count=0
            if (word in hateful_dict):
                hateful_count=int(hateful_dict[word])
            if (word in normal_dict):
                normal_count=int(normal_dict[word])
            prob_hateful = (hateful_count + 1) / (total_hateful + len(word_counts))
            prob_normal = (normal_count + 1) / (total_normal + len(word_counts))
            self.word_likelihoods[word] = (prob_hateful, prob_normal)

    def predict(self, X_test,factor):
        predictions = []
        for text in X_test:
            prob_hateful = np.log(self.class_prior[1])
            prob_normal = np.log(self.class_prior[0])
            for word, count in text.items():
                if word in self.word_likelihoods:
                    prob_hateful += count * np.log(factor*self.word_likelihoods[word][0])
                    prob_normal += count * np.log(self.word_likelihoods[word][1])
                    # print(prob_hateful)
                    # print(prob_normal)
            predictions.append(prob_normal-prob_hateful if prob_hateful > prob_normal else prob_normal-prob_hateful)
        return predictions

# Example usage:
# print(prob_hateful)
# word_counts = ["love", "hate", "awesome", "stupid"]
# hateful_counts = [1, 3, 0, 1]
# normal_counts = [2, 0, 1, 2]
word_counts=[]
for word,count in words_captions_hateful.items():
    word_counts.append(word)
for word,count in words_captions_normal.items():
    if word not in word_counts:
        word_counts.append(word)


# classifier = NaiveBayesClassifier()
# classifier.train(word_counts,words_captions_hateful, words_captions_normal)

# X_test = [{"black": 5}, {"bhai": 1, "everyone": 1}]
# predictions = classifier.predict(X_test)
# print(predictions)  # Output: [1, 0] indicating hateful and non-hateful respectively


In [228]:
import numpy as np

class GaussianNaiveBayesClassifier:
    def __init__(self):
        self.class_prior = {}
        self.word_stats = {}

    def train(self, word_counts, hateful_dict, normal_dict):
        total_hateful=0
        total_normal=0
        for val in hateful_dict.values():
            total_hateful+=val
        for val in normal_dict.values():
            total_normal+=val
        self.class_prior[1] = probab_hateful
        self.class_prior[0] = 1 - probab_hateful
        
        # self.class_prior[1] = total_hateful / (total_hateful+total_normal)
        # self.class_prior[0] = total_normal / (total_hateful+total_normal)
        
        for word in word_counts:
            hateful_count=0
            normal_count=0
            if (word in hateful_dict):
                hateful_count=int(hateful_dict[word])
            if (word in normal_dict):
                normal_count=int(normal_dict[word])
            mean_hateful = hateful_count / total_hateful
            mean_normal = normal_count / total_normal
            variance_hateful = (hateful_count * (1 - mean_hateful)**2 + (total_hateful - hateful_count) * (0 - mean_hateful)**2) / total_hateful
            variance_normal = (normal_count * (1 - mean_normal)**2 + (total_normal - normal_count) * (0 - mean_normal)**2) / total_normal
            self.word_stats[word] = (mean_hateful, variance_hateful, mean_normal, variance_normal)

    def predict(self, X_test,factor):
        predictions = []
        for text in X_test:
            prob_hateful = np.log(factor*self.class_prior[1])
            prob_normal = np.log(self.class_prior[0])
            for word, count in text.items():
                if word in self.word_stats:
                    mean_hateful, variance_hateful, mean_normal, variance_normal = self.word_stats[word]
                    prob_hateful += self.gaussian_prob(count, mean_hateful, variance_hateful)
                    prob_normal += self.gaussian_prob(count, mean_normal, variance_normal)
            predictions.append(1 if prob_hateful > prob_normal else 0)
        return predictions


    def gaussian_prob(self, x, mean, variance):
        exponent = -((x - mean) ** 2) / (2 * (variance+1e-9))
        return np.exp(exponent) / np.sqrt(2 * np.pi * (variance+1e-9))

# Example usage:
# word_counts = ["love", "hate", "awesome", "stupid"]
# hateful_counts = [1, 3, 0, 1]
# normal_counts = [2, 0, 1, 2]
X_test=[]
answers=[]
with open ("../data/train.jsonl") as f:
    for line in f: 
        elem=json.loads(line)
        temp_dict={}
        for word in elem['text'].split(" "):
            if word in temp_dict:
                temp_dict[word]+=1
            else:
                temp_dict[word]=1
        X_test.append(temp_dict)
        answers.append(elem['label'])

classifier = NaiveBayesClassifier()
classifier.train(word_counts, words_captions_hateful, words_captions_normal)

# X_test = [{"black": 100, "bitch": 100}, {"love": 1, "everyone": 1}]
# from decimal import Decimal

# low = Decimal('0.7')
# high = Decimal('1.5')
# precision = Decimal('1e-6')

# while high - low >= precision:
#     mid1 = low + (high - low) / Decimal('3')
#     mid2 = high - (high - low) / Decimal('3')
    
#     predictions1 = classifier.predict(X_test, float(mid1))
#     predictions2 = classifier.predict(X_test, float(mid2))
    
#     count1 = sum(1 for answer, predict in zip(answers, predictions1) if answer == predict)
#     count2 = sum(1 for answer, predict in zip(answers, predictions2) if answer == predict)
    
#     if count1 > count2:
#         high = mid2
#     else:
#         low = mid1

# best_factor = (low + high) / Decimal('2')

# print("Best factor:", float(best_factor))
# print("Max count:", max(count1, count2))



predictions = classifier.predict(X_test,float(1))
print(predictions)  # Output: [1, 0] indicating hateful and non-hateful respectively
count=0
count2=0
for answer,predict in zip(answers,predictions):
    if (predict<0):
        predict=1
    elif (predict>=0):
        predict=0
    if (answer==predict):
        if (answer==0):
            count+=1
        else:
            count2+=1
        # print("Haash")
        # count+=1
print(count,count2)

[-1.9823040504889207, 0.39008012662304736, 0.24725476818736425, -0.34489952653861167, -1.8764249793442431, -0.3358014972430752, 0.4202460205678591, 1.8168734178044588, -1.6320094852488225, -4.987017742737407, -2.0136117299029515, 1.1856905768861807, -0.27669574516590245, -1.6716100260421456, 2.4060930511482326, -1.4155811152573392, -2.267875134533213, 1.1469961925316454, 1.0303469160646586, -0.8051483634191072, -2.663816871656124, -2.964375559251721, -2.191402123219845, 0.1337775926764948, 0.11657344176269646, 0.12460719395505748, -2.3666604457564233, -3.4241390134413905, 0.431193409009893, -2.7730616205530083, -1.8581669808211885, 1.5793851198755533, 0.6445129053939525, 0.34192935533656055, 1.8907826855329688, -1.328913603829534, -3.3527742167911185, -0.8328672247845361, -1.6854893570659755, 0.5945748016097632, 0.8920275001240014, -1.13120339187126, 1.2770774833334855, -0.10122427288628089, -1.0686857080032048, 0.0, -1.9320280261827136, -0.15334027402529493, -2.8329875002548732, -7.41

In [185]:
import numpy as np

class NeuralNetworkClassifier:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim

        # Initialize weights and biases
        self.W1 = np.random.randn(self.input_dim, self.hidden_dim)
        self.b1 = np.zeros((1, self.hidden_dim))
        self.W2 = np.random.randn(self.hidden_dim, self.output_dim)
        self.b2 = np.zeros((1, self.output_dim))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def train(self, X_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            # Forward pass
            Z1 = np.dot(X_train, self.W1) + self.b1
            A1 = self.sigmoid(Z1)
            Z2 = np.dot(A1, self.W2) + self.b2
            A2 = self.sigmoid(Z2)

            # Compute loss
            loss = -np.mean(y_train * np.log(A2) + (1 - y_train) * np.log(1 - A2))
            
            # Backward pass
            dZ2 = A2 - y_train
            dW2 = np.dot(A1.T, dZ2) / len(X_train)
            db2 = np.sum(dZ2, axis=0, keepdims=True) / len(X_train)
            dZ1 = np.dot(dZ2, self.W2.T) * A1 * (1 - A1)
            dW1 = np.dot(X_train.T, dZ1) / len(X_train)
            db1 = np.sum(dZ1, axis=0, keepdims=True) / len(X_train)

            # Update weights and biases
            self.W1 -= learning_rate * dW1
            self.b1 -= learning_rate * db1
            self.W2 -= learning_rate * dW2
            self.b2 -= learning_rate * db2

            if epoch % 100 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')
                # pass

    def predict(self, X_test):
        Z1 = np.dot(X_test, self.W1) + self.b1
        A1 = self.sigmoid(Z1)
        Z2 = np.dot(A1, self.W2) + self.b2
        A2 = self.sigmoid(Z2)
        return A2

# Convert dictionaries to arrays
def dictionaries_to_arrays(words_captions_hateful, words_captions_normal, word_to_index):
    input_dim = len(word_to_index)
    X_train = np.zeros((len(words_captions_hateful) + len(words_captions_normal), input_dim))
    y_train = np.zeros((len(X_train), 1))

    total_hateful = sum(words_captions_hateful.values())
    total_normal = sum(words_captions_normal.values())
    # print(total_hateful)
    # print(total_normal)
    
    for i, (word, count) in enumerate(words_captions_hateful.items()):
        if word in word_to_index:
            word_index = word_to_index[word]
            X_train[i, word_index] = count
            
            relative_hateful = count / total_hateful if total_hateful != 0 else 0
            relative_normal = words_captions_normal.get(word, 0) / total_normal if total_normal != 0 else 0
            
            hateful_score = relative_hateful - relative_normal
            y_train[i] = hateful_score

    for i, (word, count) in enumerate(words_captions_normal.items()):
        if word in word_to_index:
            word_index = word_to_index[word]
            X_train[len(words_captions_hateful) + i, word_index] = count
            
            relative_hateful = words_captions_hateful.get(word, 0) / total_hateful if total_hateful != 0 else 0
            relative_normal = count / total_normal if total_normal != 0 else 0
            
            hateful_score = relative_hateful - relative_normal
            y_train[len(words_captions_hateful) + i] = hateful_score
    for word in word_to_index:
        temp_score=0
        if word in words_captions_hateful:
            temp_score-=words_captions_hateful[word]*probab_hateful_final
        if word in words_captions_normal:
            temp_score+=words_captions_normal[word]*(1-probab_hateful_final)
        # print(temp_score)
    return X_train, y_train




# Example usage:
word_counts = list(set(list(words_captions_hateful.keys()) + list(words_captions_normal.keys())))
word_to_index = {word: i for i, word in enumerate(word_counts)}
# print(word_to_index)
# print(words_captions_hateful)
X_train, y_train = dictionaries_to_arrays(words_captions_hateful, words_captions_normal, word_to_index)
# print(X_train)
input_dim = len(word_counts)
hidden_dim = 128
output_dim = 1
# print(X_test)
# print("HI")
# print(y_train)
classifier = NeuralNetworkClassifier(input_dim, hidden_dim, output_dim)
classifier.train(X_train, y_train, epochs=300, learning_rate=0.01)

# X_test = np.zeros((len(X_train), len(word_counts)))  # Example test data, modify as needed
# predictions = classifier.predict(X_test)
# print(predictions)


/tmp/ipykernel_65765/1941780568.py:16: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0, Loss: 4.481396520367307
Epoch 100, Loss: 0.20175433745215038
Epoch 200, Loss: 0.09230962599836688


In [186]:
X_train, y_train = dictionaries_to_arrays(words_captions_hateful, words_captions_normal, word_to_index)
print(X_train,y_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] [[ 6.70236378e-03]
 [ 2.92257475e-03]
 [ 1.28556255e-03]
 ...
 [-1.46603920e-05]
 [ 1.13833093e-05]
 [-1.46603920e-05]]


In [206]:
# print(words_captions_hateful)


# print(y_train)
def classify_sentence(sentence, word_to_index, classifier):
    # Tokenize the sentence and count the occurrence of each word
    print(word_to_index)
    word_counts = {}
    for word in sentence.split():
        if word in word_to_index:
            word_index = word_to_index[word]
            if word_index in word_counts:
                word_counts[word_index] += 1
            else:
                word_counts[word_index] = 1
    
    print("Word Counts:", word_counts)

    # Convert word counts to input vector
    X_test = np.zeros((1, len(word_to_index)))
    for word_index, count in word_counts.items():
        X_test[0, word_index] = count

    # print("Input Vector:", X_test)

    # # Predict the hateful scores for the words in the sentence
    predictions = classifier.predict(X_test)
    print("Predictions:", predictions)

    # Aggregate the hateful scores to classify the sentence
    hateful_score = np.sum(predictions)
    if hateful_score > 0:
        return "Hateful"
    else:
        return "Non-Hateful"

# Example usage:
sentence = "polish"
classification = classify_sentence(sentence, word_to_index, classifier)
# print(classification)
# print(X_train)
# print(y_train)
predictions = classifier.predict(X_train)
val=1.2504214
count=0
for index in range(len(predictions)):
    if (predictions[index]<=val):
        curr_prediction=0
    else:
        curr_prediction=1
    if (curr_prediction==int(labels[index])):
        count+=1
    # print(predictions[index],labels[index])
print(count/8500)
for elem in predictions:
    if (int(elem[0])<0):
        print("Hi")

{'vr': 0, 'carpet': 1, 'hyenas': 2, 'robes': 3, 'herself': 4, 'donuts': 5, 'polish': 6, 'those': 7, 'line': 8, 'chainsaw': 9, 'remote': 10, 'ornaments': 11, 'surface': 12, 'popsicle': 13, 'watering': 14, 'titles': 15, 'broom': 16, 'ants': 17, "rollin'they": 18, 'presidents': 19, 'church': 20, 'silent': 21, '10': 22, 'package': 23, 'allowed': 24, 'behind': 25, 'lighting': 26, 'tranny': 27, 'seatbelt': 28, 'job': 29, 'girlfriend': 30, 'place': 31, 'van': 32, 'expression': 33, 'lettuce,': 34, 'plan': 35, 'calling': 36, 'hat': 37, 'walkway': 38, 'football': 39, 'mulchher': 40, 'politicians': 41, 'photograph': 42, 'its': 43, 'buying': 44, 'laying': 45, 'surrounded': 46, 'trailer': 47, 'games': 48, 'unve': 49, 'trusting': 50, 'joke': 51, 'cancer': 52, 'others': 53, 'navy': 54, 'suit': 55, 'students': 56, 'ones': 57, 'mugs': 58, 'eye': 59, 'when': 60, 'true?': 61, 'die': 62, 'hacker': 63, 'form': 64, 'baby': 65, 'miss': 66, 'ethiopian': 67, 'almost': 68, 'motorized': 69, 'duping': 70, 'kerry'

/tmp/ipykernel_65765/1941780568.py:16: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))
